In [ ]:
%matplotlib inline

In [ ]:
import math
import random
from timeit import default_timer as timer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import scipy.stats as stats

# PHYS 395 - week 1

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Session 1

# Introduction to Python and the Jupyter environment

With permission from IC, I've used the import style I'm used to instead of `%pylab notebook` (I like knowing what's in my namespace).

In [ ]:
# Print "hello world"
print("hello world")

In [ ]:
# Print pi with numbers of significant digits
for i in [3, 8, 16]:
    # IC: is there a cleaner way of formatting than what I've done here?
    print("%.*f" % (i, math.pi))

# Defining functions and plotting

## The Lennard-Jones potential

The Lennard-Jones potential models interactions between pairs of neutral atoms or molecules. One equation for the Lennard-Jones potential $V_{\text{LJ}}$ is

\begin{equation}
    V_{\text{LJ}} =
        4 \epsilon
        \left(
            \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 
        \right),
\end{equation}

where $\epsilon$ is the depth of the potential well, $\sigma$ is the finite distance at which the inter-particle potential is zero, and $r$ is the distance between the particles.

Note that if we express the distance between particles $r$ in terms of $\sigma$ we can remove the $\sigma$ parameter above.

The first function below gives the LJ potential with $r$ and $\sigma$ having the same units of length. The second function gives the LJ potential where $r$ is expressed in terms of $\sigma$ (and hence does not require $\sigma$ as an argument).

In [ ]:
def lj_potential_1(r: float, sigma: float, eps: float) -> float:
    """Computes the L-J potential.

    r and sigma must be expressed in the same units of length.
    """
    l = sigma / r

    return 4 * eps * (l ** 12 - l ** 6)


def lj_potential_2(r: float, eps: float) -> float:
    """Computes the L-J potential.

    r must be expressed in terms of sigma.
    """
    return lj_potential_1(r, 1, eps)

## Plotting the LJ potential

First we will plot with $\epsilon = 1$, then we will plot with multiple values of $\epsilon$.

In [ ]:
# Note: these values are expressed in terms of sigma
r_vals = np.linspace(0.99, 15, 500)

In [ ]:
plt.plot(r_vals, [lj_potential_2(r, 1) for r in r_vals]);

This has the same shape as the classic potential curve taught in undergraduate physics.

Now we'll plot with different values of $\epsilon$. We'll keep using the same $r$ values as above.

In [ ]:
eps_vals = range(1, 4)

In [ ]:
fig, ax = plt.subplots()

# Plot
for eps in eps_vals:
    plt.plot(r_vals, [lj_potential_2(r, eps) for r in r_vals])

# Legend
ax.legend([r"$\epsilon$ = %d" % eps for eps in eps_vals])

# Labels
ax.set_xlabel(r"$r$/$\sigma$")
ax.set_ylabel(r"V/$\epsilon$")

# Save the figure
fig.savefig("ljplot.png", bbox_inches="tight")

As we increase $\epsilon$ we see that the potential energy gets more negative at its lowest point.

# Reading in data

## Trajectory analysis

The data provided has many snapshots of a droid's position (in metres). Snapshots were taken every second.

In [ ]:
# Read in CSV to a dataframe
relative_file_path = "droid_traj.csv"

df = pd.read_csv(relative_file_path, names=["x", "y"])

The data loaded in is contained in a Pandas dataframe which has 250 rows and 2 columns.

In [ ]:
num_rows, num_cols = df.shape

print("rows: %d; cols: %d" % (num_rows, num_cols))

## Plotting the trajectory

We will plot the trajectory given to us by the data.

In [ ]:
df.plot.scatter(x="x", y="y");

## Path length

We will now calculate the path length traversed for each second. The path lengths will be stored in an array, where the indices correspond to the time (0-indexed).

In [ ]:
path_lengths = np.zeros(num_rows)

for t in range(1, num_rows):
    path_lengths[t] = path_lengths[t - 1] + np.linalg.norm(df.iloc[t] - df.iloc[t - 1])

# Also create a function for this, as an alternative interface
def s(t: int) -> int:
    """Returns path length traversed at time t (0-indexed)."""
    return path_lengths[t]

The total path length is approximately 129m.

In [ ]:
print("total path length: %.2fm" % path_lengths[-1])

## Speed

We will now estimate the speed of the droid at each second. Since the time step between each data point is a second, we can simply take the difference of the path lengths.

In [ ]:
speeds = np.zeros(num_rows - 1)

for t in range(num_rows - 1):
    speeds[t] = s(t + 1) - s(t)

# Again we'll define a function for this
def v(t: int) -> int:
    """Returns estimated speed at time t (0-indexed)."""
    return speeds[t]

## Tangential acceleration

We will now estimate the tangential acceleration of the droid at each second.

In [ ]:
accels = np.zeros(num_rows - 2)

for t in range(num_rows - 2):
    accels[t] = v(t + 1) - v(t)


def a(t: int) -> int:
    """Returns estimated acceleration at time t (0-indexed)."""
    return accels[t]

## Plotting path length, speed, and tangential acceleration

Now we will plot the path length, speed, and tangential acceleration.

In [ ]:
# Create figure and axes
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(10, 15)
fig.subplots_adjust(hspace=0.25)

# Add titles
ax1.title.set_text("path length")
ax2.title.set_text("speed")
ax3.title.set_text("tangential acceleration")

# Add labels
for ax in (ax1, ax2, ax3):
    ax.set_xlabel(r"$t$ (s)")
    ax.xaxis.set_label_coords(0.5, -0.05)

ax1.set_ylabel("m")
ax2.set_ylabel("m/s")
ax3.set_ylabel(r"m/$\mathrm{s}^2$")

# Plot
ts = list(range(num_rows))
ax1.plot(ts, path_lengths)
ax2.plot(ts[:-1], speeds)
ax3.plot(ts[:-2], accels);

# Session 1 homework

## 1. Trajectory coloured by tangential acceleration

Now we will plot the droid's trajectory with the points coloured by tangential acceleration.

In [ ]:
# Trim the dataset
df_trimmed = df.drop(df.tail(2).index)

# Add in acceleration values
df_trimmed["accel"] = accels

In [ ]:
# Plot
plt.figure()

plt.scatter(x=df_trimmed["x"], y=df_trimmed["y"], c=df_trimmed["accel"], cmap="viridis")

plt.colorbar();

# Session 2

# Introduction to random numbers

## Random numbers

First we'll define a function that calculates the next element of a pseudo-random number sequence generated by a linear congruential generator.

In [ ]:
def next_element_lcg(x: int, a: int, c: int, m: int) -> int:
    """Calculates the next element of an lcg rng sequence."""
    return (a * x + c) % m

Using $x_0 = 1$, $a = 12$, $c = 0$, and $m = 143$, we will generate a sequence of 13 pseudo-random numbers.

In [ ]:
# Convenience functions
print_num = lambda x, i: print("elem %02d: %d" % (i, x))
get_next_num = lambda x: next_element_lcg(x, 12, 0, 143)

# Compute the sequence and print it
x = 1

print_num(x, 0)

for i in range(1, 14):
    x = get_next_num(x)
    print_num(x, i)

The sequence repeats with a period of 2! This is because with our choice of parameters,

\begin{align}
    (12 \cdot 1) \mod 143 &= 12, \\ \\
    (12 \cdot 12) \mod 143 &= 144 \mod 143 \\
        &= 1.
\end{align}

## The "Randu" generator

Now we will look at the "Randu" generator, which sets $a = 65539$, $c = 0$, and $m = 2^{31}$. First, we'll generate a 1000 numbers using this generator, then we'll plot the results.

In [ ]:
get_next_num = lambda x: next_element_lcg(x, 65539, 0, 2 ** 31)

# We'll start with a non-zero seed
seed = 17

# Now generate the numbers
randu_nums = np.zeros(1000)
randu_nums[0] = seed

for i in range(1, 1000):
    randu_nums[i] = get_next_num(randu_nums[i - 1])

The first plot we'll look at is a scatter plot of $x_{i + 1}$ against $x_i$.

In [ ]:
# Plot
_, ax = plt.subplots()

plt.scatter(x=randu_nums[:-1], y=randu_nums[1:])

# Labels
ax.set_xlabel(r"$x_i$")
ax.set_ylabel(r"$x_{i + 1}$");

From this relatively small set of numbers generated, everything seems fine. The points on the plot appear to uniformly cover the plane, and I can't make out any correlated bands.

Now we'll generate a 3D plot showing triplets $(x_{i + 2}, x_{i + 1}, x_i)$.

In [ ]:
# Plot
plt.figure()
ax = plt.axes(projection="3d")

ax.scatter3D(xs=randu_nums[:-2], ys=randu_nums[1:-1], zs=randu_nums[2:])

# Labels
ax.set_xlabel(r"$x_i$")
ax.set_ylabel(r"$x_{i + 1}$")
ax.set_zlabel(r"$x_{i + 2}$");

To rotate the plot change the first cell `%matplotlib inline` to `%matplotlib notebook` at the beginning of this notebook and rerun the entire notebook. However, a "pre-rotated" version of the above plot is shown below.

In [ ]:
# Plot again at a different angle
plt.figure()
ax = plt.axes(projection="3d")

ax.scatter3D(xs=randu_nums[:-2], ys=randu_nums[1:-1], zs=randu_nums[2:])

# Labels
ax.set_xlabel(r"$x_i$")
ax.set_ylabel(r"$x_{i + 1}$")
ax.set_zlabel(r"$x_{i + 2}$")

# Set the view
elev = -85.21335807050122
azim = -45.36774193548365

ax.view_init(azim=azim, elev=elev)

By rotating the plot we can see clear evidence of banding!

# Generating uniform random numbers and timing calculations in Python

First we will generate 10,000 random numbers by calling the `random.random` function 10,000 times.

In [ ]:
# Start the timer
start = timer()

# Generate the numbers
nums = [random.random() for _ in range(10 ** 4)]

# End timer and print time
end = timer()

print("time: %fs" % (end - start))

Next we'll generate 10,000 random numbers using a single call using NumPy's `random.random`.

In [ ]:
# Start the timer
start = timer()

# Generate the numbers
nums = np.random.random(10 ** 4)

# End timer and print time
end = timer()

print("time: %fs" % (end - start))

Using a single call is around an order of magnitude faster.

Now let's plot these numbers on a histogram.

In [ ]:
# Plot the histogram
plt.figure()

plt.hist(nums, bins=20);

We can see here that the numbers appear to be random. Note that if we used many more bins in the above histogram, two things would happen: (1) It would take forever to render; (2) It would highlight insignificant differences in the distribution of numbers that comes from using a small sample size (10,000, in this case).

# Probability distributions: discrete and continuous

# Radioactive decay

Let $P(t)$ be the probability density that a particle decay has not happened up to some time $t$. We are given that the probability that a decay happens in time $dt$ is given by $r \, dt$.

Given that a particle has not decayed at some time $t$, the change in probability that it has not decayed for the next time step is given by

\begin{align}
    \frac{dP}{dt} &= - \text{(probability particle decays now)}
                        \cdot \text{(probability particle hasn't decayed until now)} \\
                  &= - r P.
\end{align}

Solving for $P$, we obtain

\begin{equation}
    P(t) = c e^{-r t}
\end{equation}

for some constant $c$.

To determine what the constant $c$ is, we use the fact that $P(t)$ is a probability distribution:

\begin{align}
    \int_0^\infty P(t) dt &= \int_0^\infty c e^{-r t} dt \\
                          &= c \int_0^\infty e^{-r t} dt \\
                          &= c \left(- \frac{1}{r} \right) (0 - 1) \\
                          &= \frac{c}{r} \\
                          &= 1 \qquad \text{(P is prob. dist.)}.
\end{align}

This implies that $c = r$ and hence

\begin{equation}
    P(t) = r e^{-r t}.
\end{equation}

## Simulating decay

Now we're going to simulate 10,000 decay times using the above equation. We will choose $dt = 1$ and $r = 0.01$.

In [ ]:
def simulate_decay(r: float, dt: float) -> float:
    """Simulate decay time for a particle."""
    rdt = r * dt
    
    t = 0

    while True:
        if random.random() < rdt:
            return t

        t += dt


r = 0.01
dt = 1

# Simulate 10,000 times
decay_times = np.array([simulate_decay(r, dt) for _ in range(10 ** 4)])

Now we'll plot a plain histogram of the decay times.

In [ ]:
# Plot
_, ax = plt.subplots()

plt.hist(decay_times, bins=30)

# Labels
ax.set_xlabel(r"$t$");

Below is the same histogram, but normalized, and with the probability distribution $P(t)$ superimposed.

In [ ]:
# Plot the histogram
_, ax = plt.subplots()

plt.hist(decay_times, bins=30, density=True)

# Plot the "true" probability density
ts = np.linspace(0, 1000, 500)
plt.plot(ts, r * np.exp(-r * ts), linewidth=3)

# Labels
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"prob")

# Legend
ax.legend(["data", r"$P(t)$"]);

We see that there is strong agreement here.

## Simulating decay (again)

Here we'll do all of the above simulation with a new value of $dt = 0.01$.

In [ ]:
# Simulate 10,000 times. Note that with this value of dt,
# computing the decay times may take some time!
r = 0.01
dt = 0.01

decay_times = np.array([simulate_decay(r, dt) for _ in range(10 ** 4)])

In [ ]:
# Histogram plot
_, ax = plt.subplots()

plt.hist(decay_times, bins=30)

# Labels
ax.set_xlabel(r"$t$");

In [ ]:
# Probability density plot
_, ax = plt.subplots()

plt.hist(decay_times, bins=30, density=True)

# Plot the "true" probability density
ts = np.linspace(0, 1000, 500)
plt.plot(ts, r * np.exp(-r * ts), linewidth=3)

# Labels
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"prob")

# Legend
ax.legend(["data", r"$P(t)$"]);

With this number of bins, it's hard to see a difference in this simulation compared to the last one. However, if we increase the number of bins, we see that lowering $dt$ increases the agreement of the data with the probability density curve.

## Simulating decay with NumPy

We'll keep $r = 0.01$ as in the previous simulations.

In [ ]:
# Use NumPy exponential distribution
decay_times = np.random.exponential(scale=1 / r, size=10 ** 5)

In [ ]:
# Histogram plot
_, ax = plt.subplots()

plt.hist(decay_times, bins=30)

# Labels
ax.set_xlabel(r"$t$");

In [ ]:
# Probability density plot
_, ax = plt.subplots()

plt.hist(decay_times, bins=30, density=True)

# Plot the "true" probability density
ts = np.linspace(0, 1000, 500)
plt.plot(ts, r * np.exp(-r * ts), linewidth=3)

# Labels
ax.set_xlabel(r"$t$")
ax.set_ylabel(r"prob")

# Legend
ax.legend(["data", r"$P(t)$"]);

We see here that using the NumPy distribution directly not only agrees with our earlier results, but is faster, and more "accurate" (in the sense that we are taking the $dt \to 0$ limit).

# Random Walks

## Simulation

First we will simulate a number of 1D random walks with step size 1 for different number of steps $N$.

In [ ]:
def r(n: int, a: int = 1) -> int:
    """Calculates end-to-end distance of 1D random walk."""
    return np.sum(np.random.choice(a=[a, -a], size=n))

In [ ]:
# N values
n_vals = [5, 25, 50, 100, 200, 400, 750, 1000]

# Create an array to store results
num_ns = len(n_vals)
num_trials = 10 ** 4

data = np.zeros((num_ns, num_trials))

# Simulate the walks
for row, n in enumerate(n_vals):
    for col in range(num_trials):
        data[row][col] = r(n)

Now we'll compute the mean end-to-end distances for each $N$.

In [ ]:
# Calculate the means
mean_rs = [np.mean(data[row]) for row in range(num_ns)]

# Print the means
for idx, n in enumerate(n_vals):
    print("N = %04d;\tr(N) = %+f" % (n, mean_rs[idx]))

As was probably anticipated, the mean end-to-end distance is close to 0.

Now we will calculate the mean squared distances (MSDs) for each $N$.

In [ ]:
# Calculate MSDs
mean_sqrd_rs = [np.mean(np.square(data[row])) for row in range(num_ns)]

Now let's plot the MSDs!

In [ ]:
# Plot
_, ax = plt.subplots()

plt.scatter(x=n_vals, y=mean_sqrd_rs)

# Labels
ax.set_xlabel(r"$N$")
ax.set_ylabel("MSD");

The MSDs appear to be linear with $N$.

How might the above MSD plot look like if all steps were taken in the same direction? Well, the distance for each walk would always be $N$, so in this special case we would have

\begin{equation}
    \langle r(N)^2 \rangle = \langle N^2 \rangle = N^2
\end{equation}

## Mathematical derivations

For the means we have

\begin{align}
    \langle r(N) \rangle
        &= \sum_{i = 1}^N \langle dx_i \rangle \\
        &= \sum_{i = 1}^N \left( \frac{1}{2} a + \frac{1}{2} (-a) \right) \\
        &= \sum_{i = 1}^N 0 \\
        &= 0.
\end{align}

For the MSDs we have

\begin{equation}
    \langle r(N)^2 \rangle
        = \sum_{i = 1}^N \sum_{j = 1}^N \langle dx_i \cdot dx_j \rangle.
\end{equation}

Note that in the above sum, when $i = j$, we have

\begin{equation}
    \langle dx_i \cdot dx_j \rangle
        = \left( \frac{1}{2} a^2 + \frac{1}{2} (-a)^2 \right)
        = a^2;
\end{equation}

and for $i \neq j$,

\begin{equation}
    \langle dx_i \cdot dx_j \rangle
        = \left(
            \frac{1}{4} a^2
            + \frac{1}{4} (-a) a
            + \frac{1}{4} a (-a)
            + \frac{1}{4} (-a)^2
           \right)
        = 0.
\end{equation}

Hence,

\begin{equation}
    \langle dx_i \cdot dx_j \rangle = a^2 \delta_{ij},
\end{equation}

where $\delta_{ij}$ is the Kronecker delta function.

Returning to the MSD derivation, we have

\begin{align}
    \langle r(N)^2 \rangle
        &= \sum_{i = 1}^N \sum_{j = 1}^N \langle dx_i \cdot dx_j \rangle \\
        &= \sum_{i = 1}^N a^2 \\
        &= N a^2.
\end{align}

This agrees with the MSD scatter plot we produced above which suggested a linear relationship.

## Histograms from simulation data

In [ ]:
# Pick three Ns from our data to focus on
n1_idx = 3
n2_idx = 4
n3_idx = 5

n1, n2, n3 = n_vals[n1_idx], n_vals[n2_idx], n_vals[n3_idx]

In [ ]:
# Create figure and axes
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(10, 15)
fig.subplots_adjust(hspace=0.25)

# Add titles
for ax, n in zip((ax1, ax2, ax3), (n1, n2, n3)):
    ax.title.set_text(r"$N$ = %d" % n)

# Add labels
for ax in (ax1, ax2, ax3):
    ax.set_xlabel("distance")
    ax.set_ylabel("proportion")

# Plot
n_bins = 9

for ax, n_idx in zip((ax1, ax2, ax3), (n1_idx, n2_idx, n3_idx)):
    ax.hist(data[n_idx], bins=n_bins, density=True)

# Make limits the same
for ax in (ax1, ax2, ax3):
    ax.set_xlim(-80, 80)
    ax.set_ylim(0, 0.05)

The histograms appear to be approaching a normal distribution. Theoretically as we increase $N$, the number of steps, the histograms should get closer to the normal distribution.

We can now identify the mean $\langle r(N) \rangle$ as the mean of the normal distribution and $\langle r(N)^2 \rangle$ as the variance.

# Session 2 homework

## 1. Overlaying Gaussians on our data

For the data we have, let's overlay a Gaussian on our plots using the the mean and variance discussed above.

In [ ]:
# Create figure and axes
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(10, 15)
fig.subplots_adjust(hspace=0.25)

# Add titles
for ax, n in zip((ax1, ax2, ax3), (n1, n2, n3)):
    ax.title.set_text(r"$N$ = %d" % n)

# Add labels
for ax in (ax1, ax2, ax3):
    ax.set_xlabel("distance")
    ax.set_ylabel("proportion")

# Plot histograms
n_bins = 9

for ax, n_idx in zip((ax1, ax2, ax3), (n1_idx, n2_idx, n3_idx)):
    ax.hist(data[n_idx], bins=n_bins, density=True)

# Plot Gaussians
xs = np.linspace(-80, 80, 500)

for ax, n in zip((ax1, ax2, ax3), (n1, n2, n3)):
    ax.plot(xs, stats.norm.pdf(xs, 0, math.sqrt(n)), linewidth=3)


# Add legends
for ax in (ax1, ax2, ax3):
    ax.legend([r"$\mathcal{N}(0, N)$", "data"])

# Make limits the same
for ax in (ax1, ax2, ax3):
    ax.set_xlim(-80, 80)
    ax.set_ylim(0, 0.05)

We see in the above plots that there is good agreement between our data and the Gaussian distributions.

## 2. Simulating a 2D random walk on a square lattice

Now we will simulate a 2D random walk, where each step can be four directions (with equal weight): up, down, left, right. Here we will be interested in the means $\langle r(N) \rangle$ and mean squared end-to-end distance $\langle r(N)^2 \rangle$.

In contrast to the 1D random walk—where $r(N)$ was signed (i.e., could be positive or negative)—for the 2D random walk $r(N)$ will be the magnitude of the final displacement vector. This simplication is justified since the walk is isotropic, and hence we will have radial symmetry (in the limit of large $N$ at least).

First let's simulate the walk for a number of different $N$ values, each with multiple trials.

In [ ]:
def r_2d(n: int, a: int = 1) -> float:
    """Calculates end-to-end distance of 2D random walk."""
    return np.absolute(np.sum(np.random.choice(a=[a, -a, a * 1j, -a * 1j], size=n)))

In [ ]:
# N values
n_vals = [5, 25, 50, 100, 200, 400, 750, 1000]

# Create an array to store results
num_ns = len(n_vals)
num_trials = 10 ** 4

data = np.zeros((num_ns, num_trials))

# Simulate the walks
for row, n in enumerate(n_vals):
    for col in range(num_trials):
        data[row][col] = r_2d(n)

Now we will plot the means and MSDs.

In [ ]:
# Calculate the means
mean_rs = [np.mean(data[row]) for row in range(num_ns)]

# Calculate MSDs
mean_sqrd_rs = [np.mean(np.square(data[row])) for row in range(num_ns)]

# Plot
_, ax = plt.subplots()

plt.scatter(x=n_vals, y=mean_rs)
plt.scatter(x=n_vals, y=mean_sqrd_rs)

# Labels
ax.set_xlabel(r"$N$")

# Legend
ax.legend([r"$\langle r(N) \rangle$", r"$\langle r(N)^2 \rangle$"]);

The results are nearly identical to the results we had for the 1D random walk!